In [1]:
from pysheds.grid import Grid
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
from skimage.io import imread
import random
import numpy as np
import cv2
from PIL import Image
import os
import h5py

In [2]:
def sketch_rivers(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    depressions = grid.detect_depressions('dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    flats = grid.detect_flats('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    
    downsampled_rivers = cv2.pyrDown(np.log(grid.view('acc') + 1), dstsize=(128, 128))
    upsampled_rivers = cv2.pyrUp(downsampled_rivers, dstsize=(256, 256))
    upsampled_rivers = (upsampled_rivers - np.amin(upsampled_rivers)) / (np.amax(upsampled_rivers) - np.amin(upsampled_rivers))
    upsampled_rivers = np.array(upsampled_rivers * 255, dtype=np.uint8)
    #_, thresholded_river = cv2.threshold(upsampled_rivers, 150, 255, cv2.THRESH_BINARY)
    _, thresholded_river = cv2.threshold(upsampled_rivers, 127, 255, cv2.THRESH_BINARY)
    thresholded_river[thresholded_river == 255] = 1
    skeletonized_rivers = skeletonize(thresholded_river)
    rivers_sketch = Image.fromarray(skeletonized_rivers)
    rgb_rivers_sketch = rivers_sketch.convert('RGB')

    upsampled_depressions = np.array(depressions, dtype=np.uint8)
    upsampled_depressions = (upsampled_depressions - np.amin(upsampled_depressions)) / (np.amax(upsampled_depressions) - np.amin(upsampled_depressions))
    upsampled_depressions = np.array(upsampled_depressions * 255, dtype=np.uint8)
    _, thresholded_depressions = cv2.threshold(upsampled_depressions, 155, 255, cv2.THRESH_BINARY)
    basins_sketch = Image.fromarray(thresholded_depressions)
    rgb_basins_sketch = basins_sketch.convert('RGB')
    
    return rgb_rivers_sketch, rgb_basins_sketch

In [3]:
def sketch_ridges(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.dem = grid.dem.max() - grid.dem
    peaks = grid.detect_depressions('dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    flats = grid.detect_flats('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    downsampled_ridges = cv2.pyrDown(np.log(grid.view('acc') + 1), dstsize=(128,128))
    
    upsampled_ridges = cv2.pyrUp(downsampled_ridges, dstsize=(256, 256))
    upsampled_ridges = (upsampled_ridges - np.amin(upsampled_ridges)) / (np.amax(upsampled_ridges) - np.amin(upsampled_ridges))
    upsampled_ridges = np.array(upsampled_ridges * 255, dtype=np.uint8)
    _, thresholded_ridges = cv2.threshold(upsampled_ridges, 150, 255, cv2.THRESH_BINARY)
    thresholded_ridges[thresholded_ridges == 255] = 1
    skeletonized_ridges = skeletonize(thresholded_ridges)
    ridges_sketch = Image.fromarray(skeletonized_ridges)
    rgb_ridges_sketch = ridges_sketch.convert('RGB')
    
    upsampled_peaks = np.array(peaks, dtype=np.uint8)
    upsampled_peaks = (upsampled_peaks - np.amin(upsampled_peaks)) / (np.amax(upsampled_peaks) - np.amin(upsampled_peaks))
    upsampled_peaks = np.array(upsampled_peaks * 255, dtype=np.uint8)
    _, thresholded_peaks = cv2.threshold(upsampled_peaks, 155, 255, cv2.THRESH_BINARY)
    peaks_sketch = Image.fromarray(thresholded_peaks)
    rgb_peaks_sketch = peaks_sketch.convert('RGB')
    
    return rgb_ridges_sketch, rgb_peaks_sketch

In [4]:
def prepare_heightmap_sketch(file_path):
    rivers, basins = sketch_rivers(file_path)
    ridges, peaks = sketch_ridges(file_path)

    dim = (256, 256)

    sketch = np.zeros([256,256,3],dtype=np.uint8)
    sketch.fill(255)
    
    sketch_rivers_flag = True
    sketch_basins_flag = True
    sketch_ridges_flag = True
    sketch_peaks_flag = True
    '''
    sketch_rivers_flag = False
    sketch_basins_flag = False
    sketch_ridges_flag = False
    sketch_peaks_flag = False
    
    if random.randrange(2) == 0:
        sketch_rivers_flag = True
    
    if random.randrange(2) == 0:
        sketch_basins_flag = True
        
    if random.randrange(2) == 0:
        sketch_ridges_flag = True
        
    if random.randrange(2) == 0:
        sketch_peaks_flag = True
               
    if sketch_rivers_flag == False and sketch_basins_flag == False and sketch_ridges_flag == False and sketch_peaks_flag == False:
        sketch_rivers_flag = True
        sketch_basins_flag = True
        sketch_ridges_flag = True
        sketch_peaks_flag = True
    '''
    
    if sketch_rivers_flag == True:
        pixels_rivers = rivers.load() # create the pixel map
  
        for i in range(rivers.size[0]): # for every pixel:
            for j in range(rivers.size[1]):
                if pixels_rivers[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 0, 255)
    
    
    if sketch_basins_flag == True:
        pixels_basins = basins.load() # create the pixel map
            
        for i in range(basins.size[0]): # for every pixel:
            for j in range(basins.size[1]):
                if pixels_basins[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 255, 0)    
            
    
    if sketch_ridges_flag == True:
        pixels_ridges = ridges.load() # create the pixel map
            
        for i in range(ridges.size[0]): # for every pixel:
            for j in range(ridges.size[1]):
                if pixels_ridges[i,j] == (255, 255, 255):
                    sketch[j,i] = (255, 0, 0) 
            
    if sketch_peaks_flag == True:
        pixels_peaks = peaks.load() # create the pixel map

        for i in range(peaks.size[0]): # for every pixel:
            for j in range(peaks.size[1]):
                if pixels_peaks[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 0, 0)   
            
            
    #sketch_image = Image.fromarray(sketch)
    
    return sketch

In [5]:
files_len = len(os.listdir('heightmaps')) * 4
files_len_directory = len(os.listdir('heightmaps'))

idxs = [x for x in range(files_len)]
np.random.shuffle(idxs)

In [6]:
dim = (256, 256)

dataset_iterator = 0
iterator = 0

f = h5py.File("dataset.hdf5", "w")
f.create_dataset("sketches", (files_len, 256, 256, 3), dtype='float32')
f.create_dataset("heightmaps", (files_len, 256, 256, 3), dtype='float32')
f.create_dataset("textures", (files_len, 256, 256, 3), dtype='float32')

for iterator in range(files_len_directory):       
    print(str(dataset_iterator) + '/' + str(files_len))
    sketch = prepare_heightmap_sketch('heightmaps/topography_' + str(iterator+1) + '.tif')
    heightmap = cv2.imread('heightmaps/topography_' + str(iterator+1) + '.tif')
    texture = imread('textures/word_' + str(iterator+1) + '.png')
    
    print('heightmaps/topography_' + str(iterator+1) + '.tif')
    print('textures/word_' + str(iterator+1) + '.png')
        
    sketch_array = (sketch - 127.5) / 127.5
    heightmap_array = (np.array(heightmap) - 127.5) / 127.5
    texture_array = (np.array(texture) - 127.5) / 127.5
    
    # original rotation
    f["sketches"][idxs[dataset_iterator]] = sketch_array
    f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    f["textures"][idxs[dataset_iterator]] = texture_array
        
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    texture_array = np.rot90(texture_array)
        
    f["sketches"][idxs[dataset_iterator]] = sketch_array
    f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    texture_array = np.rot90(texture_array)
        
    f["sketches"][idxs[dataset_iterator]] = sketch_array
    f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    texture_array = np.rot90(texture_array)
        
    f["sketches"][idxs[dataset_iterator]] = sketch_array
    f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1

print(str(dataset_iterator) + '/' + str(files_len))
print("Done!")

0/21472
heightmaps/topography_1.tif
textures/word_1.png
4/21472
heightmaps/topography_2.tif
textures/word_2.png
8/21472
heightmaps/topography_3.tif
textures/word_3.png
12/21472
heightmaps/topography_4.tif
textures/word_4.png
16/21472
heightmaps/topography_5.tif
textures/word_5.png
20/21472
heightmaps/topography_6.tif
textures/word_6.png
24/21472
heightmaps/topography_7.tif
textures/word_7.png
28/21472
heightmaps/topography_8.tif
textures/word_8.png
32/21472
heightmaps/topography_9.tif
textures/word_9.png
36/21472
heightmaps/topography_10.tif
textures/word_10.png
40/21472
heightmaps/topography_11.tif
textures/word_11.png
44/21472
heightmaps/topography_12.tif
textures/word_12.png
48/21472
heightmaps/topography_13.tif
textures/word_13.png
52/21472
heightmaps/topography_14.tif
textures/word_14.png
56/21472
heightmaps/topography_15.tif
textures/word_15.png
60/21472
heightmaps/topography_16.tif
textures/word_16.png
64/21472
heightmaps/topography_17.tif
textures/word_17.png
68/21472
heightmap

heightmaps/topography_137.tif
textures/word_137.png
548/21472
heightmaps/topography_138.tif
textures/word_138.png
552/21472
heightmaps/topography_139.tif
textures/word_139.png
556/21472
heightmaps/topography_140.tif
textures/word_140.png
560/21472
heightmaps/topography_141.tif
textures/word_141.png
564/21472
heightmaps/topography_142.tif
textures/word_142.png
568/21472
heightmaps/topography_143.tif
textures/word_143.png
572/21472
heightmaps/topography_144.tif
textures/word_144.png
576/21472
heightmaps/topography_145.tif
textures/word_145.png
580/21472
heightmaps/topography_146.tif
textures/word_146.png
584/21472
heightmaps/topography_147.tif
textures/word_147.png
588/21472
heightmaps/topography_148.tif
textures/word_148.png
592/21472
heightmaps/topography_149.tif
textures/word_149.png
596/21472
heightmaps/topography_150.tif
textures/word_150.png
600/21472
heightmaps/topography_151.tif
textures/word_151.png
604/21472
heightmaps/topography_152.tif
textures/word_152.png
608/21472
heightma

heightmaps/topography_269.tif
textures/word_269.png
1076/21472
heightmaps/topography_270.tif
textures/word_270.png
1080/21472
heightmaps/topography_271.tif
textures/word_271.png
1084/21472
heightmaps/topography_272.tif
textures/word_272.png
1088/21472
heightmaps/topography_273.tif
textures/word_273.png
1092/21472
heightmaps/topography_274.tif
textures/word_274.png
1096/21472
heightmaps/topography_275.tif
textures/word_275.png
1100/21472
heightmaps/topography_276.tif
textures/word_276.png
1104/21472
heightmaps/topography_277.tif
textures/word_277.png
1108/21472
heightmaps/topography_278.tif
textures/word_278.png
1112/21472
heightmaps/topography_279.tif
textures/word_279.png
1116/21472
heightmaps/topography_280.tif
textures/word_280.png
1120/21472
heightmaps/topography_281.tif
textures/word_281.png
1124/21472
heightmaps/topography_282.tif
textures/word_282.png
1128/21472
heightmaps/topography_283.tif
textures/word_283.png
1132/21472
heightmaps/topography_284.tif
textures/word_284.png
113

heightmaps/topography_400.tif
textures/word_400.png
1600/21472
heightmaps/topography_401.tif
textures/word_401.png
1604/21472
heightmaps/topography_402.tif
textures/word_402.png
1608/21472
heightmaps/topography_403.tif
textures/word_403.png
1612/21472
heightmaps/topography_404.tif
textures/word_404.png
1616/21472
heightmaps/topography_405.tif
textures/word_405.png
1620/21472
heightmaps/topography_406.tif
textures/word_406.png
1624/21472
heightmaps/topography_407.tif
textures/word_407.png
1628/21472
heightmaps/topography_408.tif
textures/word_408.png
1632/21472
heightmaps/topography_409.tif
textures/word_409.png
1636/21472
heightmaps/topography_410.tif
textures/word_410.png
1640/21472
heightmaps/topography_411.tif
textures/word_411.png
1644/21472
heightmaps/topography_412.tif
textures/word_412.png
1648/21472
heightmaps/topography_413.tif
textures/word_413.png
1652/21472
heightmaps/topography_414.tif
textures/word_414.png
1656/21472
heightmaps/topography_415.tif
textures/word_415.png
166

heightmaps/topography_531.tif
textures/word_531.png
2124/21472
heightmaps/topography_532.tif
textures/word_532.png
2128/21472
heightmaps/topography_533.tif
textures/word_533.png
2132/21472
heightmaps/topography_534.tif
textures/word_534.png
2136/21472
heightmaps/topography_535.tif
textures/word_535.png
2140/21472
heightmaps/topography_536.tif
textures/word_536.png
2144/21472
heightmaps/topography_537.tif
textures/word_537.png
2148/21472
heightmaps/topography_538.tif
textures/word_538.png
2152/21472
heightmaps/topography_539.tif
textures/word_539.png
2156/21472
heightmaps/topography_540.tif
textures/word_540.png
2160/21472
heightmaps/topography_541.tif
textures/word_541.png
2164/21472
heightmaps/topography_542.tif
textures/word_542.png
2168/21472
heightmaps/topography_543.tif
textures/word_543.png
2172/21472
heightmaps/topography_544.tif
textures/word_544.png
2176/21472
heightmaps/topography_545.tif
textures/word_545.png
2180/21472
heightmaps/topography_546.tif
textures/word_546.png
218

...

heightmaps/topography_4825.tif
textures/word_4825.png
19300/21472
heightmaps/topography_4826.tif
textures/word_4826.png
19304/21472
heightmaps/topography_4827.tif
textures/word_4827.png
19308/21472
heightmaps/topography_4828.tif
textures/word_4828.png
19312/21472
heightmaps/topography_4829.tif
textures/word_4829.png
19316/21472
heightmaps/topography_4830.tif
textures/word_4830.png
19320/21472
heightmaps/topography_4831.tif
textures/word_4831.png
19324/21472
heightmaps/topography_4832.tif
textures/word_4832.png
19328/21472
heightmaps/topography_4833.tif
textures/word_4833.png
19332/21472
heightmaps/topography_4834.tif
textures/word_4834.png
19336/21472
heightmaps/topography_4835.tif
textures/word_4835.png
19340/21472
heightmaps/topography_4836.tif
textures/word_4836.png
19344/21472
heightmaps/topography_4837.tif
textures/word_4837.png
19348/21472
heightmaps/topography_4838.tif
textures/word_4838.png
19352/21472
heightmaps/topography_4839.tif
textures/word_4839.png
19356/21472
heightmaps

heightmaps/topography_4950.tif
textures/word_4950.png
19800/21472
heightmaps/topography_4951.tif
textures/word_4951.png
19804/21472
heightmaps/topography_4952.tif
textures/word_4952.png
19808/21472
heightmaps/topography_4953.tif
textures/word_4953.png
19812/21472
heightmaps/topography_4954.tif
textures/word_4954.png
19816/21472
heightmaps/topography_4955.tif
textures/word_4955.png
19820/21472
heightmaps/topography_4956.tif
textures/word_4956.png
19824/21472
heightmaps/topography_4957.tif
textures/word_4957.png
19828/21472
heightmaps/topography_4958.tif
textures/word_4958.png
19832/21472
heightmaps/topography_4959.tif
textures/word_4959.png
19836/21472
heightmaps/topography_4960.tif
textures/word_4960.png
19840/21472
heightmaps/topography_4961.tif
textures/word_4961.png
19844/21472
heightmaps/topography_4962.tif
textures/word_4962.png
19848/21472
heightmaps/topography_4963.tif
textures/word_4963.png
19852/21472
heightmaps/topography_4964.tif
textures/word_4964.png
19856/21472
heightmaps

heightmaps/topography_5075.tif
textures/word_5075.png
20300/21472
heightmaps/topography_5076.tif
textures/word_5076.png
20304/21472
heightmaps/topography_5077.tif
textures/word_5077.png
20308/21472
heightmaps/topography_5078.tif
textures/word_5078.png
20312/21472
heightmaps/topography_5079.tif
textures/word_5079.png
20316/21472
heightmaps/topography_5080.tif
textures/word_5080.png
20320/21472
heightmaps/topography_5081.tif
textures/word_5081.png
20324/21472
heightmaps/topography_5082.tif
textures/word_5082.png
20328/21472
heightmaps/topography_5083.tif
textures/word_5083.png
20332/21472
heightmaps/topography_5084.tif
textures/word_5084.png
20336/21472
heightmaps/topography_5085.tif
textures/word_5085.png
20340/21472
heightmaps/topography_5086.tif
textures/word_5086.png
20344/21472
heightmaps/topography_5087.tif
textures/word_5087.png
20348/21472
heightmaps/topography_5088.tif
textures/word_5088.png
20352/21472
heightmaps/topography_5089.tif
textures/word_5089.png
20356/21472
heightmaps

heightmaps/topography_5200.tif
textures/word_5200.png
20800/21472
heightmaps/topography_5201.tif
textures/word_5201.png
20804/21472
heightmaps/topography_5202.tif
textures/word_5202.png
20808/21472
heightmaps/topography_5203.tif
textures/word_5203.png
20812/21472
heightmaps/topography_5204.tif
textures/word_5204.png
20816/21472
heightmaps/topography_5205.tif
textures/word_5205.png
20820/21472
heightmaps/topography_5206.tif
textures/word_5206.png
20824/21472
heightmaps/topography_5207.tif
textures/word_5207.png
20828/21472
heightmaps/topography_5208.tif
textures/word_5208.png
20832/21472
heightmaps/topography_5209.tif
textures/word_5209.png
20836/21472
heightmaps/topography_5210.tif
textures/word_5210.png
20840/21472
heightmaps/topography_5211.tif
textures/word_5211.png
20844/21472
heightmaps/topography_5212.tif
textures/word_5212.png
20848/21472
heightmaps/topography_5213.tif
textures/word_5213.png
20852/21472
heightmaps/topography_5214.tif
textures/word_5214.png
20856/21472
heightmaps

heightmaps/topography_5325.tif
textures/word_5325.png
21300/21472
heightmaps/topography_5326.tif
textures/word_5326.png
21304/21472
heightmaps/topography_5327.tif
textures/word_5327.png
21308/21472
heightmaps/topography_5328.tif
textures/word_5328.png
21312/21472
heightmaps/topography_5329.tif
textures/word_5329.png
21316/21472
heightmaps/topography_5330.tif
textures/word_5330.png
21320/21472
heightmaps/topography_5331.tif
textures/word_5331.png
21324/21472
heightmaps/topography_5332.tif
textures/word_5332.png
21328/21472
heightmaps/topography_5333.tif
textures/word_5333.png
21332/21472
heightmaps/topography_5334.tif
textures/word_5334.png
21336/21472
heightmaps/topography_5335.tif
textures/word_5335.png
21340/21472
heightmaps/topography_5336.tif
textures/word_5336.png
21344/21472
heightmaps/topography_5337.tif
textures/word_5337.png
21348/21472
heightmaps/topography_5338.tif
textures/word_5338.png
21352/21472
heightmaps/topography_5339.tif
textures/word_5339.png
21356/21472
heightmaps

In [7]:
f.close()